# Running your first-level analysis

Now it's time for the fun stuff! Our first step in fMRI analysis is to perform a *first-level analysis* for each of our participants. This is basically just a regression --- we are testing the fit of a GLM against the time series of signal change in each individual voxel of the participant's brain during the task we are interested in.

This notebook differs a bit from the past three in that there is no one script that does everything presented here. First-level analysis is something you'll probably be doing a *lot* of, and it also takes up a *lot* of space on the server. So instead what you'll see here are just a few examples of the ways it can be done. Those examples can also be accessed in the terminal as: 

1. B3/scripts/step_four_a_example_one_run_contrast_decon.sh
2. B3/scripts/step_four_a_example_two_run_contrast_decon.sh
3. B3/scripts/step_four_a_example_two_run_pmod_decon.sh

I haven't included a one_run_pmod example because my hope is that after seeing the contrast examples you'll get the idea. Then you'll build your own scripts for whatever contrasts you want to test (note that you can run as many first-level analyses as you want in a single script... it will just take up a lot of time and space. 

### Prerequisite steps:

By the time you get here, you should have:
* Completed the steps in the Step ONE, TWO, and THREE notebooks for all participants of interest
* Checked to ensure that you're not almost out of server space (type 'df' in the command line and pause if the DOP-Restricted drive is at more than 90% capacity)

#### Note: 

This notebook documents the process I use in AFNI. It's proven good enough for publication in some journals (as of 2023/24), but it is by no means the only way or even the ideal way to run this analysis. To my knowledge it has no fatal flaws, but researchers aiming for prestige journals (Nature, Science, etc.) might find that this doesn't check all of the boxes reviewers are hoping for. One area where I KNOW it falls short of the cutting edge is in the number of nuisance regressors included -- the ones here are standard, but some hard-nosed reviewers could ask for more. Again, this is definitely good enough for JoC and even many nuero journals, but just something to keep in mind as we set our sights higher in the future. 

## Step 1: Modeling with 3dDeconvovle

The first step is to build our model and test its fit! The scripts below give examples of three types of models you're likely to run. 

### Testing a contrast on participants with one run 

The code below will test for differences in the BOLD signal (a measure of the amount of deoxygenated blood) across two different conditions. This is what you'll use 90% of the time. 

This code is designed for analysis of those participants who only have ONE run for a given task. This matters because these participants have fewer motion files to include in the model (a clever coder could probably fix that by editing the preprocessing scripts a little, but I don't think it's necessary to). So when building your subject list, you'll want to included only those participants. For the purpose of this example, I've only included one participant. 

It uses an AFNI command called 3dDeconvolve, the documentation for which can be found [here](https://afni.nimh.nih.gov/pub/dist/doc/program_help/3dDeconvolve.html).

This example compares signal during the viewing of positive peer feedback (peer_like) with viewing of negative peer feedback (peer_dislike). Broken down, here what we're doing:


1. First we create the loop that iterates through all the subjects. Then we assign the path for our files (the participant's func directory) to a variable just to save space. 


2. The **-input** argument describes the files that we'll be running this regression on. We used the scaled files that we created after fmriprep.


3. **-mask** specifies a file to use as a mask of the participant's brain, so we are only running analysis on signal change in parts of the brain (not skull, empty space, etc.). We use the mask of the brain after it has been transformed to the MNI-152 template. 


4. **-polort** specifies a polynomial for use in filtering out drift in the signal. We use "A" because that chooses this number for us automatically. 


5. **-num_stimts** specifies the number of timing files we'll be including in the regression. This needs to be correct of the script will crash. 


6. **-stim_times** specifies the timing files for our contrasts of interest. Those are the files you created in the last step. **note the use of 'BLOCK(8,1)' here. The first number specifies that our stimulus duration is 8 seconds long. THIS IS NOT THE SAME FOR EVERY TASK -- MAKE SURE THAT NUMBER IS CORRECT. The second number specifies the hypothisized amplitude of the signal, and should pretty much always be 1**. 


7. **-stim_label** provides a label for that stim file. Give it a number and a name. 


8. **-stim_file** is just a slightly different argument that we use to load in the motion regressors. Those regressors should follow the file format you see here for every run of every task. Don't forget to add the right numbers and labels for -stim_base and stim_label here. 


9. **-jobs** communicates to the HTPC that it should use 8 CPUS to process this. I honestly have no idea if this is the optimal number for this system, but it doesn't impact the analysis -- only the speed of processing. 


10. **-gltsym** specifies the relationship you're interested in between your contrasts of interest. Use the stim_label value for those two. Usually "SYM: (label 1) -(label 2)" is correct **note that there is a space after the first label but not after the minus sign. This matters.**


11. **-glt_label** labels this regressor that you're using to estimate your contrast of interest. Use the format you see here. 


12. **-fout** **-tout** and everything here exports some files that illustrate the model. You'll want to look at these later. 


13. **-bucket** creates the file that contains all of the info from your statistical tests for each regressor. This is the file we'll be most interest in. 


14. **-fitts** creates a file that contains info about the fit of the full model


15. -errts creates a file that contains info about the residuals from your model

You should be able to just run this --- just don't forget to include the \ at the end of each line when writing your own script, and don't put any blank spaces on the right side of the backslash. 

In [25]:
%%bash

for subj in 447; do

dir=../fmriprep_outputs/sub-${subj}/ses-01/func/

3dDeconvolve -input $dir/sub-${subj}_ses-01_task-peerfeedback_run-0*_scale.nii                                      \
    -mask $dir/sub-${subj}_ses-01_task-peerfeedback_run-01_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz         \
    -polort A                                                                                                       \
    -num_stimts 8                                                                                                   \
    -stim_times 1 ../timings/onerun1D/${subj}.peerfeedback_peer_like.1D 'BLOCK(8,1)' -stim_label 1 peer_like                     \
    -stim_times 2 ../timings/onerun1D/${subj}.peerfeedback_peer_dislike.1D 'BLOCK(8,1)' -stim_label 2 peer_dislike               \
    -stim_file 3  $dir/sub-${subj}_ses-01_task-peerfeedback_run-01_rot_x.txt -stim_base 3 -stim_label 3 rot_x_01    \
    -stim_file 4  $dir/sub-${subj}_ses-01_task-peerfeedback_run-01_rot_y.txt -stim_base 4 -stim_label 4 rot_y_01    \
    -stim_file 5 $dir/sub-${subj}_ses-01_task-peerfeedback_run-01_rot_z.txt -stim_base 5 -stim_label 5 rot_z_01     \
    -stim_file 6 $dir/sub-${subj}_ses-01_task-peerfeedback_run-01_trans_x.txt -stim_base 6 -stim_label 6 trans_x_01 \
    -stim_file 7 $dir/sub-${subj}_ses-01_task-peerfeedback_run-01_trans_y.txt -stim_base 7 -stim_label 7 trans_y_01 \
    -stim_file 8 $dir/sub-${subj}_ses-01_task-peerfeedback_run-01_trans_z.txt -stim_base 8 -stim_label 8 trans_z_01 \
    -jobs 8                                                                                                         \
    -gltsym 'SYM: peer_like -peer_dislike'     -glt_label 1 peer_like-peer_dislike                                  \
    -fout -tout -x1D ../outputs/${subj}.peerfeedback_peer_like-peer_dislike.1D                                      \
    -xjpeg ../outputs/${subj}.X.peerfeedback_peer_like-peer_dislike.jpg                                             \
    -bucket ../outputs/stats.${subj}.peerfeedback_peer_like-peer_dislike                                            \
    -fitts ../outputs/fitts.${subj}.peerfeedback_peer_like-peer_dislike                                             \
    -errts ../outputs/errts.${subj}.peerfeedback_peer_like-peer_dislike

done


------------------------------------------------------------
GLT matrix from 'SYM: peer_like -peer_dislike':
   0   0   0   0   0   0   1  -1   0   0   0   0   0   0 
 


++ 3dDeconvolve: AFNI version=AFNI_23.0.07 (Mar  1 2023) [64-bit]
++ Authored by: B. Douglas Ward, et al.
++ loading dataset ../fmriprep_outputs/sub-447/ses-01/func//sub-447_ses-01_task-peerfeedback_run-01_scale.nii
++ Skipping check for initial transients
++ Imaging duration=664.0 s; Automatic polort=5
++ -stim_times using TR=1 s for stimulus timing conversion
++ -stim_times using TR=1 s for any -iresp output datasets
++  [you can alter the -iresp TR via the -TR_times option]
++ ** -stim_times NOTE ** guessing GLOBAL times if 1 time per line; LOCAL otherwise
++ ** GUESSED ** -stim_times 1 using LOCAL times
++ ** GUESSED ** -stim_times 2 using LOCAL times
++ Number of time points: 664 (no censoring)
 + Number of parameters:  14 [12 baseline ; 2 signal]
++ total shared memory needed = 2,915,737,488 bytes (about 2.9 billion)
++ mmap() memory allocated: 2,915,737,488 bytes (about 2.9 billion)
++ Memory required for output bricks = 2,915,737,488 bytes (about 2.9 billion)
++ Wrote matrix im

You should see an output that contains a whole lot of information. Most of this is not extremely important, but keep an eye out for warnings --- these are usually a sign that something is wrong with one of the regressor files. If the warning is that a few of the timings are outside the input time series, check the log file to see if the scan stopped short or the task ran long. 

### Testing a contrast on participants with two runs

I won't go into nearly as much depth with this one, because it's pretty much the exact same script. You'll just notice that we've changed the path for our timing files, added a few extra stim files (the motion time series for run 2), and updated our num_stimts argument. 

In [29]:
%%bash

for subj in 177 342 476; do

dir=../fmriprep_outputs/sub-${subj}/ses-01/func/

3dDeconvolve -input $dir/sub-${subj}_ses-01_task-peerfeedback_run-0*_scale.nii                                         \
    -mask $dir/sub-${subj}_ses-01_task-peerfeedback_run-01_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz            \
    -polort A                                                                                                          \
    -num_stimts 14                                                                                                      \
    -stim_times 1 ../timings/tworun1D/${subj}.peerfeedback_peer_like.1D 'BLOCK(8,1)' -stim_label 1 peer_like           \
    -stim_times 2 ../timings/tworun1D/${subj}.peerfeedback_peer_dislike.1D 'BLOCK(8,1)' -stim_label 2 peer_dislike     \
    -stim_file 3  $dir/sub-${subj}_ses-01_task-peerfeedback_run-01_rot_x.txt -stim_base 3 -stim_label 3 rot_x_01       \
    -stim_file 4  $dir/sub-${subj}_ses-01_task-peerfeedback_run-01_rot_y.txt -stim_base 4 -stim_label 4 rot_y_01       \
    -stim_file 5 $dir/sub-${subj}_ses-01_task-peerfeedback_run-01_rot_z.txt -stim_base 5 -stim_label 5 rot_z_01        \
    -stim_file 6 $dir/sub-${subj}_ses-01_task-peerfeedback_run-01_trans_x.txt -stim_base 6 -stim_label 6 trans_x_01    \
    -stim_file 7 $dir/sub-${subj}_ses-01_task-peerfeedback_run-01_trans_y.txt -stim_base 7 -stim_label 7 trans_y_01    \
    -stim_file 8 $dir/sub-${subj}_ses-01_task-peerfeedback_run-01_trans_z.txt -stim_base 8 -stim_label 8 trans_z_01    \
    -stim_file 9  $dir/sub-${subj}_ses-01_task-peerfeedback_run-02_rot_x.txt -stim_base 9 -stim_label 9 rot_x_02       \
    -stim_file 10  $dir/sub-${subj}_ses-01_task-peerfeedback_run-02_rot_y.txt -stim_base 10 -stim_label 10 rot_y_02    \
    -stim_file 11 $dir/sub-${subj}_ses-01_task-peerfeedback_run-02_rot_z.txt -stim_base 11 -stim_label 11 rot_z_02     \
    -stim_file 12 $dir/sub-${subj}_ses-01_task-peerfeedback_run-02_trans_x.txt -stim_base 12 -stim_label 12 trans_x_02 \
    -stim_file 13 $dir/sub-${subj}_ses-01_task-peerfeedback_run-02_trans_y.txt -stim_base 13 -stim_label 13 trans_y_02 \
    -stim_file 14 $dir/sub-${subj}_ses-01_task-peerfeedback_run-02_trans_z.txt -stim_base 14 -stim_label 14 trans_z_02 \
    -jobs 8                                                                                                            \
    -gltsym 'SYM: peer_like -peer_dislike'     -glt_label 1 peer_like-peer_dislike                                     \
    -fout -tout -x1D ../outputs/${subj}.peerfeedback_peer_like-peer_dislike.1D                                         \
    -xjpeg ../outputs/${subj}.X.peerfeedback_peer_like-peer_dislike.jpg                                                \
    -bucket ../outputs/stats.${subj}.peerfeedback_peer_like-peer_dislike                                               \
    -fitts ../outputs/fitts.${subj}.peerfeedback_peer_like-peer_dislike                                                \
    -errts ../outputs/errts.${subj}.peerfeedback_peer_like-peer_dislike

done

------------------------------------------------------------
GLT matrix from 'SYM: peer_like -peer_dislike':
   0   0   0   0   0   0   0   0   1  -1   0   0   0   0   0   0   0   0   0   0   0   0 
 


++ 3dDeconvolve: AFNI version=AFNI_23.0.07 (Mar  1 2023) [64-bit]
++ Authored by: B. Douglas Ward, et al.
++ loading dataset ../fmriprep_outputs/sub-177/ses-01/func//sub-177_ses-01_task-peerfeedback_run-01_scale.nii ../fmriprep_outputs/sub-177/ses-01/func//sub-177_ses-01_task-peerfeedback_run-02_scale.nii
++ Auto-catenated input datasets treated as multiple imaging runs
++ Auto-catenated datasets start at:  0 348
++ Skipping check for initial transients
++ Imaging duration=348.0 s; Automatic polort=3
++ -stim_times using TR=1 s for stimulus timing conversion
++ -stim_times using TR=1 s for any -iresp output datasets
++  [you can alter the -iresp TR via the -TR_times option]
++ ** -stim_times NOTE ** guessing GLOBAL times if 1 time per line; LOCAL otherwise
++ ** GUESSED ** -stim_times 1 using LOCAL times
++ ** GUESSED ** -stim_times 2 using LOCAL times
++ Number of time points: 673 (no censoring)
 + Number of parameters:  22 [20 baseline ; 2 signal]
++ total shared memory needed = 2,95

### Testing for parametric modulation on participants with two runs 

You might also want to run a parametric modulation analysis, which tests whether the amplitude of the BOLD signal varies as a function of some other factor, like peer feedback or responses in the button box. 

The code below tests whether the BOLD signal varies as a linear function of peer feedback, in a participant who did two runs of the peer feedback task. It's a lot like the code we ran above, but differs in a few important ways: 

1. We have one fewer stim time, because we aren't contrasting two runs. 


2. We present our stim times with the argument **-stim_times_AM2**, which lets 3dDeconvolve know that there is additiona information in the file that needs to be considered


3. We replaced our BLOCK(8,1) argument with GAM, which is more approrpriate for parametric modulation


4. We removed the **gltsym** argument, because we don't need it! 


5. We renamed output files, of course --- never forget to do this if you are copying and pasting code!


We'll run this for three participants. 

In [37]:
%%bash

for subj in 177 342 476; do

dir=../fmriprep_outputs/sub-${subj}/ses-01/func/

3dDeconvolve -input $dir/sub-${subj}_ses-01_task-peerfeedback_run-0*_scale.nii                                         \
    -mask $dir/sub-${subj}_ses-01_task-peerfeedback_run-01_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz            \
    -polort A                                                                                                          \
    -num_stimts 13                                                                                                     \
    -stim_times_AM2 1 ../timings/tworun1D/${subj}.peerfeedback_feedback_pmod.1D 'GAM' -stim_label 1 feedback_pmod      \
    -stim_file 2  $dir/sub-${subj}_ses-01_task-peerfeedback_run-01_rot_x.txt -stim_base 2 -stim_label 2 rot_x_01       \
    -stim_file 3  $dir/sub-${subj}_ses-01_task-peerfeedback_run-01_rot_y.txt -stim_base 3 -stim_label 3 rot_y_01       \
    -stim_file 4 $dir/sub-${subj}_ses-01_task-peerfeedback_run-01_rot_z.txt -stim_base 4 -stim_label 4 rot_z_01        \
    -stim_file 5 $dir/sub-${subj}_ses-01_task-peerfeedback_run-01_trans_x.txt -stim_base 5 -stim_label 5 trans_x_01    \
    -stim_file 6 $dir/sub-${subj}_ses-01_task-peerfeedback_run-01_trans_y.txt -stim_base 6 -stim_label 6 trans_y_01    \
    -stim_file 7 $dir/sub-${subj}_ses-01_task-peerfeedback_run-01_trans_z.txt -stim_base 7 -stim_label 7 trans_z_01    \
    -stim_file 8  $dir/sub-${subj}_ses-01_task-peerfeedback_run-02_rot_x.txt -stim_base 8 -stim_label 8 rot_x_02       \
    -stim_file 9  $dir/sub-${subj}_ses-01_task-peerfeedback_run-02_rot_y.txt -stim_base 9 -stim_label 9 rot_y_02       \
    -stim_file 10 $dir/sub-${subj}_ses-01_task-peerfeedback_run-02_rot_z.txt -stim_base 10 -stim_label 10 rot_z_02     \
    -stim_file 11 $dir/sub-${subj}_ses-01_task-peerfeedback_run-02_trans_x.txt -stim_base 11 -stim_label 11 trans_x_02 \
    -stim_file 12 $dir/sub-${subj}_ses-01_task-peerfeedback_run-02_trans_y.txt -stim_base 12 -stim_label 12 trans_y_02 \
    -stim_file 13 $dir/sub-${subj}_ses-01_task-peerfeedback_run-02_trans_z.txt -stim_base 13 -stim_label 13 trans_z_02 \
    -jobs 8                                                                                                            \
    -fout -tout -x1D ../outputs/${subj}.peerfeedback_feedback_pmod.1D                                         \
    -xjpeg ../outputs/${subj}.X.peerfeedback_feedback_pmod.jpg                                                \
    -bucket ../outputs/stats.${subj}.peerfeedback_feedback_pmod                                               \
    -fitts ../outputs/fitts.${subj}.peerfeedback_feedback_pmod                                                \
    -errts ../outputs/errts.${subj}.peerfeedback_feedback_pmod

done

++ '-stim_times_AM2 1 ../timings/tworun1D/177.peerfeedback_feedback_pmod.1D' has 1 auxiliary values per time point
++ '-stim_times_AM2 1 ../timings/tworun1D/177.peerfeedback_feedback_pmod.1D' will have 2 regressors
++ 3dDeconvolve: AFNI version=AFNI_23.0.07 (Mar  1 2023) [64-bit]
++ Authored by: B. Douglas Ward, et al.
++ loading dataset ../fmriprep_outputs/sub-177/ses-01/func//sub-177_ses-01_task-peerfeedback_run-01_scale.nii ../fmriprep_outputs/sub-177/ses-01/func//sub-177_ses-01_task-peerfeedback_run-02_scale.nii
++ Auto-catenated input datasets treated as multiple imaging runs
++ Auto-catenated datasets start at:  0 348
++ Skipping check for initial transients
++ Imaging duration=348.0 s; Automatic polort=3
++ -stim_times using TR=1 s for stimulus timing conversion
++ -stim_times using TR=1 s for any -iresp output datasets
++  [you can alter the -iresp TR via the -TR_times option]
++ ** -stim_times NOTE ** guessing GLOBAL times if 1 time per line; LOCAL otherwise
++ ** GUESSED ** -

## Step 2: Reviewing the output

Next, we'll look at the outputs of our analysis just to make sure nothing screwy happened. 

### Looking at the models
First, we'll use the code below to pull up an image that illustrates our two-run contrast model. When you run the code, an image should pop up on the screen. 


In [1]:
%%bash

1dplot ../outputs/476.peerfeedback_peer_like-peer_dislike.1D

++ 1dplot: AFNI version=AFNI_23.0.07 (Mar  1 2023) [64-bit]
++ Authored by: RWC et al.
Error: Can't open display: 


CalledProcessError: Command 'b'\n1dplot ../outputs/476.peerfeedback_peer_like-peer_dislike.1D\n'' returned non-zero exit status 1.

Ok! Here we can see all of the time series we are using as regressors in our model. 

* The first 12 are our motion time series. You'll notice that each is fully flat for one half of the scan --- that's because these files only cover one of the two runs. If any one run looks suspiciously flat all around (all of our trans_ time series look that way), you might want to open up the .txt file itself. You'll probably see that the numbers there are just very small (if you're looking at a run 2 file, remember that you'll need to scroll through hundreds of zeros to get to any numbers --- that's normal!). If ALL numbers are 0, something has gone wrong in preprocessing. 


* The next two time series are our contrasts. You'll see a little bump for each presentation of a stimulus that meets that category --- in this case the top time series represents time participants saw positive peer feedback. The important thing here is to make sure these aren't overlapping -- your contrasts should be mutually exclusive. 


* The final time series are all there to account for drift in signal. They should look like this every time -- just make sure they are there. 

Ok -- now let's look at an illustration of our parametric modulation model. 

In [22]:
%%bash

1dplot ../outputs/476.peerfeedback_feedback_pmod.1D

++ 1dplot: AFNI version=AFNI_23.0.07 (Mar  1 2023) [64-bit]
++ Authored by: RWC et al.


Everything here should look the same, except for the stimulus time series contrasts. The top contrast models our parametric modulation regressor, which predicts that signal during each stimulus presentation will be some degree higher or lower than the mean, depending on the modulator. The time series below it assumes a mean amplitude for every stimulus. 

### Checking the sub-bricks

Next, I like to make sure that all of my regressors are stored in the stats file in the way I'd expect. The code below should do that for the contrast file:

In [23]:
%%bash 

3dinfo -verb ../outputs/stats.476.peerfeedback.peer_like-peer_dislike+tlrc.


Dataset File:    stats.476.peerfeedback.peer_like-peer_dislike+tlrc
Identifier Code: AFN_qG7grv9soVCLePkebxBg1w  Creation Date: Thu Oct 24 20:29:26 2024
Template Space:  MNI
Dataset Type:    Func-Bucket (-fbuc)
Byte Order:      LSB_FIRST [this CPU native = LSB_FIRST]
Storage Mode:    BRIK
Storage Space:   21,791,760 (22 million) bytes
Geometry String: "MATRIX(-2.5,0,0,76,0,-1.875,0,111.375,0,0,1.875,-76.125):62,101,87"
Data Axes Tilt:  Plumb
Data Axes Orientation:
  first  (x) = Left-to-Right
  second (y) = Posterior-to-Anterior
  third  (z) = Inferior-to-Superior   [-orient LPI]
R-to-L extent:   -76.500 [R] -to-    76.000 [L] -step-     2.500 mm [ 62 voxels]
A-to-P extent:   -76.125 [A] -to-   111.375 [P] -step-     1.875 mm [101 voxels]
I-to-S extent:   -76.125 [I] -to-    85.125 [S] -step-     1.875 mm [ 87 voxels]
Number of values stored at each pixel = 10
  -- At sub-brick #0 'Full_Fstat' datum type is float:            0 to       196.717
     statcode = fift;  statpar = 2 651
  

++ 3dinfo: AFNI version=AFNI_23.0.07 (Mar  1 2023) [64-bit]


The important information here is the content of the sub-bricks. You'll see that the coefficient (Coef) for our peer_like-peer_dislike regressor is in **sub-brick 7**. That's where it should be, and we'll need that information later. 

If you're running a contrast analysis and your contrast regressor *isn't* at sub-brick 7, you've probably done something wrong. Specifically, you probably made an error in assigning the numbers to your motion regressors (check the -stim_base argument). 

Now let's look at the same info for our parametric modulation model. 

In [24]:
%%bash

3dinfo -verb ../outputs/stats.476.peerfeedback_feedback_pmod+tlrc.


Dataset File:    stats.476.peerfeedback_feedback_pmod+tlrc
Identifier Code: AFN_d2lVT316_F6h64BroKoRZg  Creation Date: Thu Oct 24 20:16:09 2024
Template Space:  MNI
Dataset Type:    Func-Bucket (-fbuc)
Byte Order:      LSB_FIRST [this CPU native = LSB_FIRST]
Storage Mode:    BRIK
Storage Space:   13,075,056 (13 million) bytes
Geometry String: "MATRIX(-2.5,0,0,76,0,-1.875,0,111.375,0,0,1.875,-76.125):62,101,87"
Data Axes Tilt:  Plumb
Data Axes Orientation:
  first  (x) = Left-to-Right
  second (y) = Posterior-to-Anterior
  third  (z) = Inferior-to-Superior   [-orient LPI]
R-to-L extent:   -76.500 [R] -to-    76.000 [L] -step-     2.500 mm [ 62 voxels]
A-to-P extent:   -76.125 [A] -to-   111.375 [P] -step-     1.875 mm [101 voxels]
I-to-S extent:   -76.125 [I] -to-    85.125 [S] -step-     1.875 mm [ 87 voxels]
Number of values stored at each pixel = 6
  -- At sub-brick #0 'Full_Fstat' datum type is float:            0 to       287.819
     statcode = fift;  statpar = 2 651
  -- At sub-

++ 3dinfo: AFNI version=AFNI_23.0.07 (Mar  1 2023) [64-bit]


We have fewer sub-bricks here --- note that our coefficient of interest (feedback_pmod) is at **sub-brick 3**. That's where it ought to be. 

Now we can go on to the next bit --- pulling the ROIs for analysis in R.  

## Step 3: Pulling ROI data

Finally, let's pull the data from our regions or networks of interest. This part of the code *is* contained in its own script at B3/scripts/step_four_b_roi_puller.sh.

This code pulls data from the regions of interest that we've stored in B3/ROIs. There are many ways to create or download new ROIs, but I've tried to store all the ones we used in past studies. 

This code creates a dataframe that can be analyzed in R. Basically it does the following: 

1. Deletes any "betas" files that have been created by past ROI pulls. These files are easy to make, and it can cause lots of problems when they overwrite each other. So I prefer to just start from a clean slate every time. 


2. Writes a set of column headers to a .csv. These should include the name of the regressor and the name of the ROI (for example 'peerlike_peerdislike_neurosynth_conflict'). You should edit these headers and the .csv name every time you make a new dataset. 


3. Iterates over all participants whose subject IDs are in the subject list. 


4. Writes a file that contains the coefficient of interest (from the sub-brick we identified above) for each voxel in the brain. 


5. Checks to make sure that coefficient exists for the given participant (some participants may be missing from certain contrasts)


6. If the participant has that coefficient, averages the coefficients of all voxels within a specific region and assigns that number to a variable. 


7. If the participant does *not* have that coefficient, assigns the value "NA" to that variable.


8. Prints a row to the .csv. 

We'll do this for four participants that we ran earlier. 

In [39]:
%%bash

echo "Participant, peer_like-peer_dislike_neurosynth_conflict, peer_like-peer_dislike_neurosynth_self-referential", "feedback_pmod_neurosynth_conflict", "feedback_pmod_neurosynth_conflict" >> ../Betas/example_ROIs.csv

rm ../outputs/*Betas*

for subj in 177 342 476 447; do

3dbucket -aglueto ../outputs/${subj}.peerfeedback_peer_like-peer_dislike_Betas+tlrc.HEAD ../outputs/stats.${subj}.peerfeedback_peer_like-peer_dislike+tlrc.'[7]' # This calls the sub-brick we checked earlier    
3dbucket -aglueto ../outputs/${subj}.peerfeedback_feedback_pmod_Betas+tlrc.HEAD ../outputs/stats.${subj}.peerfeedback_feedback_pmod+tlrc.'[3]'

if [ -e "../outputs/${subj}.peerfeedback_peer_like-peer_dislike_Betas+tlrc.HEAD" ]; then
  LDC=$(3dmaskave -quiet -mask ../ROIs/neurosynth_conflict+tlrc.BRIK ../outputs/${subj}.peerfeedback_peer_like-peer_dislike_Betas+tlrc)
  LDS=$(3dmaskave -quiet -mask ../ROIs/neurosynth_self-referential+tlrc.BRIK ../outputs/${subj}.peerfeedback_peer_like-peer_dislike_Betas+tlrc)
  
else 
  LDC='NA'
  LDS='NA'

fi

if [ -e "../outputs/${subj}.peerfeedback_feedback_pmod_Betas+tlrc.HEAD" ]; then
  FBC=$(3dmaskave -quiet -mask ../ROIs/neurosynth_conflict+tlrc.BRIK ../outputs/${subj}.peerfeedback_feedback_pmod_Betas+tlrc.)
  FBS=$(3dmaskave -quiet -mask ../ROIs/neurosynth_self-referential+tlrc.BRIK ../outputs/${subj}.peerfeedback_feedback_pmod_Betas+tlrc)

else
  FBC='NA'
  FBS='NA'
  
fi

echo "${subj},${LDC},${LDS},${FBC},${FBS}" >> ../Betas/example_ROIs.csv

done

++ 3dbucket: AFNI version=AFNI_23.0.07 (Mar  1 2023) [64-bit]
++ 3dbucket: AFNI version=AFNI_23.0.07 (Mar  1 2023) [64-bit]
++ 3dmaskave: AFNI version=AFNI_23.0.07 (Mar  1 2023) [64-bit]
+++ 1023 voxels survive the mask
++ 3dmaskave: AFNI version=AFNI_23.0.07 (Mar  1 2023) [64-bit]
+++ 3465 voxels survive the mask
++ 3dmaskave: AFNI version=AFNI_23.0.07 (Mar  1 2023) [64-bit]
+++ 1023 voxels survive the mask
++ 3dmaskave: AFNI version=AFNI_23.0.07 (Mar  1 2023) [64-bit]
+++ 3465 voxels survive the mask
++ 3dbucket: AFNI version=AFNI_23.0.07 (Mar  1 2023) [64-bit]
++ 3dbucket: AFNI version=AFNI_23.0.07 (Mar  1 2023) [64-bit]
++ 3dmaskave: AFNI version=AFNI_23.0.07 (Mar  1 2023) [64-bit]
+++ 1023 voxels survive the mask
++ 3dmaskave: AFNI version=AFNI_23.0.07 (Mar  1 2023) [64-bit]
+++ 3465 voxels survive the mask
++ 3dmaskave: AFNI version=AFNI_23.0.07 (Mar  1 2023) [64-bit]
+++ 1023 voxels survive the mask
++ 3dmaskave: AFNI version=AFNI_23.0.07 (Mar  1 2023) [64-bit]
+++ 3465 voxels s

Cool! Now navigate to B3/Betas and open the example_ROIs.csv file in excel. You should see fairly small values in most cells, and NA values in the cells for 447's parametric modulation. That's because 447 was the one-run participant who we only ran the contrast analysis on. 

#### Note: 
Remember -- these values represent the average coefficient for the given regressor across each ROI. That means, for example, that for participant 177 about .07% of the change in BOLD signal within the self-referential ROI can be explained by whether the participant is viewing positive or negative feedback. Knowing this, you should never see values in these cells that are larger than one. If you do, chances are that you have pulled your betas from the wrong sub-brick. 

## That's it! 

Congrats --- you've now pulled your first set of ROIs! 

That's about as far as I can take you with these notebooks, but it's just the first step of your long journey into the world of fMRI analysis. That journey will definitely have its share of frustration, self-doubt, and buggy code (more than its fair share of that, actually), but in time it will lead you to some fascinating and unexpected places.

Some resources I've found extremely helpful along the way are:

[Andy's brain book](https://andysbrainbook.readthedocs.io/en/latest/) -- an excellent resource for beginners. Andrew Jahn's [youtube channel](https://www.youtube.com/@AndrewJahn) has step-by-step tutorials for many of the analyses you'll want to run. 

The [AFNI message board](https://discuss.afni.nimh.nih.gov/) is a great resource for answering most debugging questions.

[Neurostars](https://neurostars.org/) is THE forum for neuroscience research. It's a great place to catch up on pretty much everything. 

[The Transmitter](https://www.thetransmitter.org/) isn't really an analysis resource, but it's a nice publication that keeps up with the cutting edge of neuroscience.

...and there's still me, of course. Presuming you're reading this in 2024/25, you should still be able to reach me with the occassional question via email at mminich@wisc.edu. 

Happy coding!

Thanks and cheers, 

Matt